In [1]:
import matplotlib.pyplot as plt
from holisticai.datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from holisticai.xai.commons._feature_importance import permutation_feature_importance, filter_feature_importance
from holisticai.xai.commons._partial_dependence import compute_partial_dependence
from holisticai.xai.metrics.global_importance import AlphaImportanceScore, XAIEaseScore, PositionParity, RankAlignment

ImportError: cannot import name 'accuracy_score' from 'sklearn.base' (/home/cristian/.local/share/hatch/env/virtual/holisticai/g8_gwgw3/testing/lib/python3.11/site-packages/sklearn/base.py)

In [2]:
# Dataset
dataset = load_dataset('adult')
dataset = dataset.train_test_split(test_size=0.2, random_state=42)
train = dataset['train']
test = dataset['test']

In [ ]:
model = LogisticRegression() # instantiate model
model.fit(train['X'], train['y']) # fit model

y_pred = model.predict(test['X']) # compute predictions

# compute efficacy metrics
accuracy_score(test['y'], y_pred)

# Global Explainability Metrics (based on Permutation Feature Importance)

In [ ]:
feature_importance = permutation_feature_importance(predict_fn=model.predict, X_test=test['X'], y_test=test['y'], metric=accuracy_score)
features = feature_importance.Variable.tolist()

conditional_feature_importance = {
    group_name[0]:permutation_feature_importance(predict_fn=model.predict, X_test=group_test['X'], y_test=group_test['y'], metric=accuracy_score)
    for group_name, group_test in test.groupby('y')
}

conditional_feature = {
    group_name:cfi.Variable.tolist()
    for group_name, cfi in conditional_feature_importance.items()
}

ranked_features_importances = filter_feature_importance(feature_importance, alpha=0.8)
ranked_features = ranked_features_importances.Variable.tolist()

pdep = compute_partial_dependence(test['X'], 
                                  features=ranked_features, 
                                  percentiles=(0.05, 0.95), 
                                  learning_task="binary_classification", 
                                  predict_fn=model.predict, 
                                  predict_proba_fn=model.predict_proba)

# Alpha Importance Score

In [ ]:
metric = AlphaImportanceScore()
metric(feature_importance, ranked_features_importances), metric.reference

# XAI Ease Score

In [ ]:
metric = XAIEaseScore()
metric(pdep, ranked_features), metric.reference

# Position Parity

In [ ]:
metric = PositionParity()
metric(conditional_feature, ranked_features),  metric.reference

# Rank Alignment

In [ ]:
metric = RankAlignment()
metric(conditional_feature, ranked_features),  metric.reference